# Import Libraries

In [49]:
import pandas as pd
import os
import re
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import requests
import csv
from io import StringIO
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Loading

In [14]:
def data_loading(root, category):
    root_path = "dataset"
    folder_path = root_path + f"/{category}"

    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv") and f!=f"{category}.csv"]

    # List to store each DataFrame
    dfs = []

    # Read and collect DataFrames
    for file in csv_files:
        full_path = os.path.join(folder_path, file)
        # print(f"Reading: {full_path}")
        df = pd.read_csv(full_path)
        # print(df.head(5))
        dfs.append(df)
    
    # Combine all into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Show result
    # print(combined_df.head())
    print(f"\nTotal combined rows of {category}: {len(combined_df)}")

    return combined_df

In [15]:
root_path = "dataset"

# List all CSV files in the folder
categories = [f for f in os.listdir(root_path)]

print(categories)

['Elektronik', 'Aksesoris', 'Kesehatan', 'Kecantikan', 'Alas_kaki', 'Pakaian_Pria', 'Pakaian_Wanita']


In [ ]:
category_names = [
    "elektronik",
    "aksesoris",
    "kesehatan",
    "kecantikan",
    "alas_kaki",
    "pakaian_pria",
    "pakaian_wanita"
]

all_dfs = {
    name: data_loading(root_path, category)
    for name, category in zip(category_names, categories)
}



Total combined rows of Elektronik: 6287

Total combined rows of Aksesoris: 9659

Total combined rows of Kesehatan: 2676

Total combined rows of Kecantikan: 17285

Total combined rows of Alas_kaki: 6969

Total combined rows of Pakaian_Pria: 11537

Total combined rows of Pakaian_Wanita: 7670


In [20]:
all_dfs['elektronik'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6287 entries, 0 to 6286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  6086 non-null   object
 1   rating  6287 non-null   object
 2   date    4447 non-null   object
dtypes: object(3)
memory usage: 147.5+ KB


# Data Preprocessing

### Helper Functions

In [21]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
 
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks

    # remove emojis
    emoji_pattern = re.compile(
        "["                               
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002700-\U000027BF"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    
    text = emoji_pattern.sub(r'', text)
    
    return text
 
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text
 
# def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
#     text = word_tokenize(text)
#     return text
 
def tokenizingText(text):
    if pd.isna(text):
        return []
    return text.split()

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    # listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords)
    # listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','nya','na','sih','ku',"di","ya","loh","kah","woi","woii","woy", "nih", "trus", "tuh",\
                          "yah", "ajah", "lagi", "lah", "aj", "aja", "jg", "juga", "jga", "yng", 'apa', "cuman", "deh",\
                            "min", "gak"\
                            "si", "an"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    # Memecah teks menjadi daftar kata
    words = text.split()
 
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
 
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [32]:
# Download slang dictionary from a public dataset (if available)
url = "https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/refs/heads/master/combined_slang_words.txt"
slang_dict = requests.get(url).json()

slang_dict['aku'] = 'saya'
slang_dict['ak'] = 'saya'
slang_dict['gua'] = 'saya'
slang_dict['gw'] = 'saya'

slang_dict['jlk'] = 'jelek'
slang_dict['jlek'] = 'jelek'
slang_dict['burik'] = 'jelek'
slang_dict['buriq'] = 'jelek'
slang_dict['ampas'] = 'jelek'
slang_dict['amps'] = 'jelek'
slang_dict['buruk'] = 'jelek'
slang_dict['kentang'] = 'jelek'
slang_dict['bobrok'] = 'jelek'

slang_dict['bgs'] = 'bagus'
slang_dict['bgus'] = 'bagus'
slang_dict['baguss'] = 'bagus'

slang_dict['trnyata'] = 'ternyata'

slang_dict['bgt'] = 'banget'

slang_dict['kren'] = 'keren'

slang_dict['udh'] = 'udah'

slang_dict['kasi'] = 'kasih'
slang_dict['ksi'] = 'kasih'
slang_dict['ksih'] = 'kasih'

slang_dict['gk'] = 'gak'
slang_dict['ga'] = 'gak'
slang_dict['gaa'] = 'gak'
slang_dict['kagak'] = 'gak'
slang_dict['kgk'] = 'gak'
slang_dict['g'] = 'gak'
slang_dict['engga'] = 'gak'
slang_dict['tdk'] = 'gak'
slang_dict['nggk'] = 'gak'

slang_dict['jls'] = 'jelas'
slang_dict['jlas'] = 'jelas'
slang_dict['danta'] = 'jelas'

slang_dict['mntp'] = 'mantap'
slang_dict['mantul'] = 'mantap'
slang_dict['mntap'] = 'mantap'

slang_dict['lg'] = 'lagi'
slang_dict['lgi'] = 'lagi'

slang_dict['uk'] = 'ukuran'

slang_dict['ksel'] = 'kesal'
slang_dict['kesel'] = 'kesal'
slang_dict['sebel'] = 'kesal'
slang_dict['sebal'] = 'kesal'

slang_dict['bacod'] = 'bacot'
slang_dict['bct'] = 'bacot'
slang_dict['bcd'] = 'bacot'

slang_dict['goblog'] = 'goblok'
slang_dict['gblg'] = 'goblok'
slang_dict['gblk'] = 'goblok'
slang_dict['bego'] = 'goblok'
slang_dict['bgo'] = 'goblok'
slang_dict['tolol'] = 'goblok'
slang_dict['tlol'] = 'goblok'
slang_dict['idiot'] = 'goblok'

slang_dict['trun'] = 'turun'

slang_dict['brg'] = 'barang'

slang_dict['yt'] = 'youtube'

slang_dict['ajg'] = 'anjing'
slang_dict['anj'] = 'anjing'
slang_dict['anjg'] = 'anjing'
slang_dict['anjir'] = 'anjing'
slang_dict['anjr'] = 'anjing'

slang_dict['lemod'] = 'lemot'

slang_dict['pass'] = 'pas'

slang_dict['gaje'] = 'aneh'

slang_dict['wr'] = 'win rate'
slang_dict['winrate'] = 'win rate'
slang_dict['ws'] = 'win streak'
slang_dict['winstreak'] = 'win streak'

slang_dict['leg'] = 'lag'
slang_dict['ngeleg'] = 'ngelag'

slang_dict['hps'] = 'hapus'
slang_dict['hpus'] = 'hapus'
slang_dict['uninstal'] = 'hapus'
slang_dict['uninstall'] = 'hapus'

# Bi-gram
slang_dict['gak jelas'] = 'aneh'
slang_dict['ga jelas'] = 'aneh'
slang_dict['ngga jelas'] = 'aneh'
slang_dict['g jelas'] = 'aneh'

def fix_slangwords(text):
    words = text.lower().split()
    
    # Step 1: Fix unigrams
    fixed_unigrams = [slang_dict.get(word, word) for word in words]
    
    # Step 2: Check for fixed bigrams
    i = 0
    final_words = []
    while i < len(fixed_unigrams):
        if i + 1 < len(fixed_unigrams):
            bigram = f"{fixed_unigrams[i]} {fixed_unigrams[i+1]}"
            if bigram in slang_dict:
                final_words.append(slang_dict[bigram])
                i += 2
                continue
        
        final_words.append(fixed_unigrams[i])
        i += 1

    return ' '.join(final_words)

In [23]:
def preprocess_review(review_df):
    # Remove missing values before pre-processing
    review_df.dropna(subset=['review'], inplace=True)

    # Membersihkan teks dan menyimpannya di kolom 'text_clean'
    review_df['text_clean'] = review_df['review'].apply(cleaningText)
    
    # Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
    review_df['text_casefoldingText'] = review_df['text_clean'].apply(casefoldingText)
    
    # Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
    review_df['text_slangwords'] = review_df['text_casefoldingText'].apply(fix_slangwords)

    # Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
    review_df['text_tokenizingText'] = review_df['text_slangwords'].apply(tokenizingText)
    
    # Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
    review_df['text_stopword'] = review_df['text_tokenizingText'].apply(filteringText)
    
    # Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
    review_df['text_akhir'] = review_df['text_stopword'].apply(toSentence)

    return review_df

In [24]:
for name, df in all_dfs.items():
    print(f"Category: {name}")
    df = preprocess_review(df)

Category: elektronik
Category: aksesoris
Category: kesehatan
Category: kecantikan
Category: alas_kaki
Category: pakaian_pria
Category: pakaian_wanita


In [25]:
all_dfs['kecantikan'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,"brg tiba tepat waktu , bahan ringan lgsg menya...",bintang 5,NaN,brg tiba tepat waktu bahan ringan lgsg menyat...,brg tiba tepat waktu bahan ringan lgsg menyat...,bareng tiba tepat waktu bahan ringan langsung ...,"[bareng, tiba, tepat, waktu, bahan, ringan, la...","[bareng, bahan, ringan, langsung, menyatu, kul...",bareng bahan ringan langsung menyatu kulitwrna...
1,terimakasi mas/mba cushionnya udah sampe denga...,bintang 5,NaN,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya sudah sampai deng...,"[terimakasi, masmba, cushionnya, sudah, sampai...","[terimakasi, masmba, cushionnya, suka, warnany...",terimakasi masmba cushionnya suka warnanya bagus
2,"suka sama cushion nya, pas di warna, ringan ba...",bintang 5,NaN,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,"[suka, sama, cushion, nya, pas, di, warna, rin...","[suka, cushion, pas, warna, ringan, banget, ku...",suka cushion pas warna ringan banget kulit rev...
3,Desainnya lucu. Penjualnya ramah banget. Packa...,bintang 5,NaN,Desainnya lucu Penjualnya ramah banget Packagi...,desainnya lucu penjualnya ramah banget packagi...,desainnya lucu penjualnya ramah banget packagi...,"[desainnya, lucu, penjualnya, ramah, banget, p...","[desainnya, lucu, penjualnya, ramah, banget, p...",desainnya lucu penjualnya ramah banget packagi...
4,Gampang dipakai. Desainnya mewah. Proses pesan...,bintang 5,NaN,Gampang dipakai Desainnya mewah Proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,"[gampang, dipakai, desainnya, mewah, proses, p...","[gampang, dipakai, desainnya, mewah, proses, p...",gampang dipakai desainnya mewah proses pesanan...


# Labeling

In [26]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [37]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
 
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    # Inisialisasi skor sentimen ke 0
 
    for word in text:
        # Mengulangi setiap kata dalam teks
 
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]/5.
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen
 
    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)
 
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]/5.
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen
 
    polarity=''
    # Inisialisasi variabel polaritas
 
    if (score >= 0.2):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < -1.0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
        polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan
 
    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [38]:
def labeling(preprocessed_df):
    results = preprocessed_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
    results = list(zip(*results))
    preprocessed_df['polarity_score'] = results[0]
    preprocessed_df['label'] = results[1]

In [39]:
for name, df in all_dfs.items():
    print(f"Category: {name}")
    df = labeling(df)

Category: elektronik
Category: aksesoris
Category: kesehatan
Category: kecantikan
Category: alas_kaki
Category: pakaian_pria
Category: pakaian_wanita


In [40]:
all_dfs['kecantikan'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,label
0,"brg tiba tepat waktu , bahan ringan lgsg menya...",bintang 5,NaN,brg tiba tepat waktu bahan ringan lgsg menyat...,brg tiba tepat waktu bahan ringan lgsg menyat...,bareng tiba tepat waktu bahan ringan langsung ...,"[bareng, tiba, tepat, waktu, bahan, ringan, la...","[bareng, bahan, ringan, langsung, menyatu, kul...",bareng bahan ringan langsung menyatu kulitwrna...,-0.2,neutral
1,terimakasi mas/mba cushionnya udah sampe denga...,bintang 5,NaN,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya sudah sampai deng...,"[terimakasi, masmba, cushionnya, sudah, sampai...","[terimakasi, masmba, cushionnya, suka, warnany...",terimakasi masmba cushionnya suka warnanya bagus,0.0,neutral
2,"suka sama cushion nya, pas di warna, ringan ba...",bintang 5,NaN,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,"[suka, sama, cushion, nya, pas, di, warna, rin...","[suka, cushion, pas, warna, ringan, banget, ku...",suka cushion pas warna ringan banget kulit rev...,2.0,positive
3,Desainnya lucu. Penjualnya ramah banget. Packa...,bintang 5,NaN,Desainnya lucu Penjualnya ramah banget Packagi...,desainnya lucu penjualnya ramah banget packagi...,desainnya lucu penjualnya ramah banget packagi...,"[desainnya, lucu, penjualnya, ramah, banget, p...","[desainnya, lucu, penjualnya, ramah, banget, p...",desainnya lucu penjualnya ramah banget packagi...,2.4,positive
4,Gampang dipakai. Desainnya mewah. Proses pesan...,bintang 5,NaN,Gampang dipakai Desainnya mewah Proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,"[gampang, dipakai, desainnya, mewah, proses, p...","[gampang, dipakai, desainnya, mewah, proses, p...",gampang dipakai desainnya mewah proses pesanan...,2.0,positive


In [41]:
all_dfs['kecantikan'][all_dfs['kecantikan']['label'] == 'negative'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,label
46,Bagus bgt…dapat sticker juga…aku kira keputiha...,bintang 5,NaN,Bagus bgtdapat sticker jugaaku kira keputihan ...,bagus bgtdapat sticker jugaaku kira keputihan ...,bagus bgtdapat sticker jugaaku kira keputihan ...,"[bagus, bgtdapat, sticker, jugaaku, kira, kepu...","[bagus, bgtdapat, sticker, jugaaku, keputihan,...",bagus bgtdapat sticker jugaaku keputihan bgtta...,-1.2,negative
88,"pengiriman cepat, sayang pengemasan kurang ama...",bintang 5,NaN,pengiriman cepat sayang pengemasan kurang aman...,pengiriman cepat sayang pengemasan kurang aman...,pengiriman cepat sayang pengemasan kurang aman...,"[pengiriman, cepat, sayang, pengemasan, kurang...","[pengiriman, cepat, sayang, pengemasan, aman, ...",pengiriman cepat sayang pengemasan aman pembun...,-1.6,negative
107,pengiriman tumben agak lama. ternyta warna ini...,bintang 4,NaN,pengiriman tumben agak lama ternyta warna ini ...,pengiriman tumben agak lama ternyta warna ini ...,pengiriman tumben agak lama ternyta warna ini ...,"[pengiriman, tumben, agak, lama, ternyta, warn...","[pengiriman, tumben, ternyta, warna, kuning, a...",pengiriman tumben ternyta warna kuning arah ny...,-3.4,negative
127,"bagus, tapi aku salah ambil shade 😅\nthank u",bintang 5,NaN,bagus tapi aku salah ambil shade thank u,bagus tapi aku salah ambil shade thank u,bagus tapi saya salah ambil shade thank kamu,"[bagus, tapi, saya, salah, ambil, shade, thank...","[bagus, salah, ambil, shade, thank]",bagus salah ambil shade thank,-1.4,negative
169,Bagus sih buat pemula tapi gak waterfop cepet ...,bintang 5,5 bulan lalu,Bagus sih buat pemula tapi gak waterfop cepet ...,bagus sih buat pemula tapi gak waterfop cepet ...,bagus sih buat pemula tapi gak waterfop cepat ...,"[bagus, sih, buat, pemula, tapi, gak, waterfop...","[bagus, pemula, gak, waterfop, cepat, hilang, ...",bagus pemula gak waterfop cepat hilang gesek t...,-1.4,negative


In [42]:
all_dfs['kecantikan'][all_dfs['kecantikan']['label'] == 'neutral'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,label
0,"brg tiba tepat waktu , bahan ringan lgsg menya...",bintang 5,NaN,brg tiba tepat waktu bahan ringan lgsg menyat...,brg tiba tepat waktu bahan ringan lgsg menyat...,bareng tiba tepat waktu bahan ringan langsung ...,"[bareng, tiba, tepat, waktu, bahan, ringan, la...","[bareng, bahan, ringan, langsung, menyatu, kul...",bareng bahan ringan langsung menyatu kulitwrna...,-0.2,neutral
1,terimakasi mas/mba cushionnya udah sampe denga...,bintang 5,NaN,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya udah sampe dengan...,terimakasi masmba cushionnya sudah sampai deng...,"[terimakasi, masmba, cushionnya, sudah, sampai...","[terimakasi, masmba, cushionnya, suka, warnany...",terimakasi masmba cushionnya suka warnanya bagus,0.0,neutral
5,"packaging aman, pengemasan cepat, pengiriman j...",bintang 5,NaN,packaging aman pengemasan cepat pengiriman jug...,packaging aman pengemasan cepat pengiriman jug...,packaging aman pengemasan cepat pengiriman jug...,"[packaging, aman, pengemasan, cepat, pengirima...","[packaging, aman, pengemasan, cepat, pengirima...",packaging aman pengemasan cepat pengiriman cep...,-1.0,neutral
8,baguss banget paketnya aman gk ada lecet sama ...,bintang 5,NaN,baguss banget paketnya aman gk ada lecet sama ...,baguss banget paketnya aman gk ada lecet sama ...,bagus banget paketnya aman gak ada lecet sama ...,"[bagus, banget, paketnya, aman, gak, ada, lece...","[bagus, banget, paketnya, aman, gak, lecet, be...",bagus banget paketnya aman gak lecet bermasala...,-1.0,neutral
10,"Barang okee, packing baik, pengiriman cepat. S...",bintang 5,NaN,Barang okee packing baik pengiriman cepat Shad...,barang okee packing baik pengiriman cepat shad...,barang okee packing baik pengiriman cepat shad...,"[barang, okee, packing, baik, pengiriman, cepa...","[barang, okee, packing, pengiriman, cepat, sha...",barang okee packing pengiriman cepat shade ses...,-0.4,neutral


In [43]:
all_dfs['kecantikan'][all_dfs['kecantikan']['label'] == 'positive'].head()

,review,rating,date,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,label
2,"suka sama cushion nya, pas di warna, ringan ba...",bintang 5,NaN,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,suka sama cushion nya pas di warna ringan bang...,"[suka, sama, cushion, nya, pas, di, warna, rin...","[suka, cushion, pas, warna, ringan, banget, ku...",suka cushion pas warna ringan banget kulit rev...,2.0,positive
3,Desainnya lucu. Penjualnya ramah banget. Packa...,bintang 5,NaN,Desainnya lucu Penjualnya ramah banget Packagi...,desainnya lucu penjualnya ramah banget packagi...,desainnya lucu penjualnya ramah banget packagi...,"[desainnya, lucu, penjualnya, ramah, banget, p...","[desainnya, lucu, penjualnya, ramah, banget, p...",desainnya lucu penjualnya ramah banget packagi...,2.4,positive
4,Gampang dipakai. Desainnya mewah. Proses pesan...,bintang 5,NaN,Gampang dipakai Desainnya mewah Proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,gampang dipakai desainnya mewah proses pesanan...,"[gampang, dipakai, desainnya, mewah, proses, p...","[gampang, dipakai, desainnya, mewah, proses, p...",gampang dipakai desainnya mewah proses pesanan...,2.0,positive
6,penjual memproses dan mengirimkan produk denga...,bintang 5,NaN,penjual memproses dan mengirimkan produk denga...,penjual memproses dan mengirimkan produk denga...,penjual memproses dan mengirimkan produk denga...,"[penjual, memproses, dan, mengirimkan, produk,...","[penjual, memproses, mengirimkan, produk, cepa...",penjual memproses mengirimkan produk cepat pac...,1.8,positive
7,"Produk original, harga terjangkau, pengemasan ...",bintang 5,NaN,Produk original harga terjangkau pengemasan cepat,produk original harga terjangkau pengemasan cepat,produk original harga terjangkau pengemasan cepat,"[produk, original, harga, terjangkau, pengemas...","[produk, original, harga, terjangkau, pengemas...",produk original harga terjangkau pengemasan cepat,1.2,positive


In [44]:
# Drop the unused columns
for name, df in all_dfs.items():
    print(f"Category: {name}")
    df = df.drop(columns=['polarity_score', 'text_clean', 'text_casefoldingText', 'text_slangwords', 'text_tokenizingText', 'text_stopword', 'review'])
    
# labeled_df = preprocessed_df.drop(columns=['text_clean', 'text_casefoldingText', 'text_slangwords', 'text_tokenizingText', 'text_stopword', 'review'])

# labeled_df.head()

Category: elektronik
Category: aksesoris
Category: kesehatan
Category: kecantikan
Category: alas_kaki
Category: pakaian_pria
Category: pakaian_wanita


### Checking the label distribution

In [45]:
for name, df in all_dfs.items():
    print(f"Category: {name}")
    print(df['label'].value_counts())

Category: elektronik
label
positive    3731
neutral     2095
negative     260
Name: count, dtype: int64
Category: aksesoris
label
positive    4267
neutral     3482
negative     351
Name: count, dtype: int64
Category: kesehatan
label
positive    1438
neutral     1009
negative     160
Name: count, dtype: int64
Category: kecantikan
label
positive    7420
neutral     6048
negative     761
Name: count, dtype: int64
Category: alas_kaki
label
positive    2968
neutral     2108
negative     214
Name: count, dtype: int64
Category: pakaian_pria
label
positive    5625
neutral     4502
negative     697
Name: count, dtype: int64
Category: pakaian_wanita
label
neutral     3216
positive    2623
negative     721
Name: count, dtype: int64


### Balancing DF

In [51]:
def balance_df(df):
    # Separate by class
    df_0 = df[df['label'] == 'negative']
    df_1 = df[df['label'] == 'neutral']
    df_2 = df[df['label'] == 'positive']

    # Find the smallest class size
    min_class_size = min(len(df_0), len(df_1), len(df_2))

    # Generate random multipliers for each class
    mult_0 = np.random.uniform(1.0, 1.5)
    mult_1 = np.random.uniform(1.0, 1.5)
    mult_2 = np.random.uniform(1.0, 1.5)

    # Compute target sample sizes
    n_0 = min(len(df_0), int(min_class_size * mult_0))
    n_1 = min(len(df_1), int(min_class_size * mult_1))
    n_2 = min(len(df_2), int(min_class_size * mult_2))

    # Downsample each class
    df_0_down = resample(df_0, replace=False, n_samples=n_0, random_state=42)
    df_1_down = resample(df_1, replace=False, n_samples=n_1, random_state=43)
    df_2_down = resample(df_2, replace=False, n_samples=n_2, random_state=44)

    # Combine and shuffle
    df_balanced = pd.concat([df_0_down, df_1_down, df_2_down])
    df_balanced = df_balanced.sample(frac=1, random_state=45).reset_index(drop=True)

    # View new distribution
    print("Random multipliers:", f"Class 0: {mult_0:.2f}, Class 1: {mult_1:.2f}, Class 2: {mult_2:.2f}")
    print(df_balanced['label'].value_counts())

    return df_balanced

In [52]:
for name, df in all_dfs.items():
    print(f"Category: {name}")
    df = balance_df(df)

Category: elektronik
Random multipliers: Class 0: 1.17, Class 1: 1.38, Class 2: 1.43
label
positive    372
neutral     359
negative    260
Name: count, dtype: int64
Category: aksesoris
Random multipliers: Class 0: 1.31, Class 1: 1.06, Class 2: 1.24
label
positive    434
neutral     372
negative    351
Name: count, dtype: int64
Category: kesehatan
Random multipliers: Class 0: 1.17, Class 1: 1.36, Class 2: 1.34
label
neutral     217
positive    215
negative    160
Name: count, dtype: int64
Category: kecantikan
Random multipliers: Class 0: 1.04, Class 1: 1.49, Class 2: 1.26
label
neutral     1132
positive     959
negative     761
Name: count, dtype: int64
Category: alas_kaki
Random multipliers: Class 0: 1.12, Class 1: 1.45, Class 2: 1.35
label
neutral     310
positive    289
negative    214
Name: count, dtype: int64
Category: pakaian_pria
Random multipliers: Class 0: 1.45, Class 1: 1.18, Class 2: 1.06
label
neutral     823
positive    741
negative    697
Name: count, dtype: int64
Category

# Feature Engineering